# This notebook is used to run the funEstimate function from MEGSA to compute pvalues

In [1]:
import numpy as np
import scipy.stats as sp
import pandas as pd
import os

import rpy2
from rpy2 import robjects as r
from rpy2.robjects import pandas2ri

In [2]:
pwd

'E:\\pancancer_stages_epistasis\\version yiii - MEGSA\\version beta'

### Cancer Types: ['BLCA', 'BRCA', 'COADREAD', 'LUAD', 'LUSC', 'SKCM', 'STAD', 'UCEC']
### Threshold parameters: 5,10,20

In [3]:
cancer_type = 'BRCA'
threshold = 20
# infile = 'mutationMat_LAML.txt' #for test
infile = '../data/binary_matrices_all_genes_ep_mutation_filtered/{}_TML_binary_sm.txt'.format(cancer_type)
outpath = '../out/megsa_mutation_filtered_ep_data/'
outfile = outpath + '{}_megsa_result_mutations_all_genes_normal_{}.txt'.format(cancer_type,threshold)
outfile_intact = outpath + '{}_pairs_normal_intact_filtered_subset{}.txt'.format(cancer_type,threshold)

if not os.path.exists(outpath):
    os.makedirs(outpath)
    

In [4]:
## Load df, drop column named y (placeholder for TML value)

df = pd.read_csv(infile, sep='\t', header=0,index_col=0)
df.drop('y',1, inplace=True)
df.head()

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patients,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-3C-AALI-01A,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TCGA-3C-AALJ-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-3C-AALK-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-4H-AAAK-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Filter dataframe based on threshold Values

In [5]:
col_list = []
for col in df.columns:
#     print(col, sum(df[col]))
    if sum(df[col])>threshold:
        col_list.append(col)

df2 = df[col_list]
df2

,ABCA13,ABCA8,ADAMTSL3,AFF2,AHCTF1,AHNAK,AHNAK2,AK9,AKAP9,AKT1,...,VPS13D,WDFY3,WDFY4,WDR44,WNK3,XIRP2,ZAN,ZFHX3,ZFHX4,ZNF292
patients,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-3C-AALI-01A,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
TCGA-3C-AALJ-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
TCGA-3C-AALK-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-4H-AAAK-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WT-AB44-01A,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
TCGA-XX-A899-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-XX-A89A-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Run R script

In [6]:
## convert python pd dataframe to r dataframe  
pandas2ri.activate()
rdf_in = pandas2ri.py2ri(df2)
rdf_in

ABCA13,ABCA8,ADAMTSL3,...,ZFHX3,ZFHX4,ZNF292
0,0,0,...,0,0,0
0,1,0,...,0,1,0
0,0,0,...,0,1,0
0,0,0,...,0,0,0
...,...,...,...,...,...,...
0,0,0,...,0,0,0
0,0,0,...,0,0,0
0,0,0,...,0,0,0
0,0,0,...,0,0,0


In [7]:
# run R sctipt from this notebook. Make sure to have the correct R filename
r.r.source('run_megsa.R')
rdf = r.r['run_megsa'](rdf_in, detail=False)
rdf


KeyboardInterrupt: 

In [ ]:
# convert R df to pd df
df_temp = r.conversion.ri2py(rdf)
df_temp['gene1'] = df_temp['gene1'].str.upper()
df_temp['gene2'] = df_temp['gene2'].str.upper()
df_temp

In [ ]:
## Write to file
df_temp.to_csv(outfile, sep='\t')

## Intact filtering
The mutual exclusivity values are filtered on Intact PPI

In [ ]:
intact_edge_file = '../data/intact_nodupl_edge_file.txt'
intact_index_file = '../data/intact_nodupl_index_file.txt'

with open(intact_index_file, 'r') as f:
    indices = {line.split()[0]:line.split()[1] for line in f.readlines()}

with open(intact_edge_file, 'r') as f:
    edges = [(indices[line.split()[0]].upper(),indices[line.split()[1]].upper()) for line in f.readlines()]
len(edges)

intact_genes_list = list(indices.values())
intact_genes_list[:5]

In [ ]:
cols = df_temp.columns.to_list()

g1_list = df_temp['gene1'].tolist()
g2_list = df_temp['gene2'].tolist()
llk = df_temp['loglikelyhood'].tolist()
pval = df_temp['pvalue'].tolist()

##CCreate dictionary for lookup
dict_temp = {g1+'\t'+g2:(p,q) for g1,g2,p,q in zip(g1_list,g2_list,llk,pval)}

print('dict_temp_done')

list_temp = []
#edges should have no duplicates
for e in edges:
    g1 = e[0]
    g2 = e[1]
    
    if '{}\t{}'.format(g1,g2) in dict_temp:
        llk,pval = dict_temp['{}\t{}'.format(g1,g2)]
        list_temp.append([g1,g2,llk,pval])
        
    elif '{}\t{}'.format(g2,g1) in dict_temp:
        llk,pval = dict_temp['{}\t{}'.format(g2,g1)]
        list_temp.append([g1,g2,llk,pval])
         
print(len(list_temp))
df_intact = pd.DataFrame(list_temp,columns=cols)
df_intact

In [ ]:
df_intact.to_csv(outfile_intact, sep='\t')